In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

import networkx as nx
import matplotlib.pyplot as plt
import tensorly


2022-09-16 09:22:24.029701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%run -i simulator.py

In [3]:
Ncommunities = 2

B = tfp.distributions.Beta(1/10, 1/2).sample((Ncommunities,Ncommunities))
B = tf.experimental.numpy.triu(B + 0.5*tf.eye(Ncommunities))
B = B + tf.transpose(tf.experimental.numpy.triu(B, k = 1))
B

2022-09-16 09:22:36.569129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.5000488 , 0.02227278],
       [0.02227278, 0.77789843]], dtype=float32)>

In [4]:
initial_distribution = tfp.distributions.Dirichlet(np.abs(np.random.normal(5, 0.1, Ncommunities))).sample()
initial_distribution

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.3851048, 0.6148952])>

In [5]:
N = 100

simulator = stochasticBlockModel(B)
E         = simulator.pop_sample(initial_distribution, N)

A = simulator.sample(E)

In [6]:
def d_spectral_tensorly(A, d):

    eigenvectors, eigenvalues, _ = tensorly.partial_svd(A.numpy(), n_eigenvecs=d)

    eigenvalues_sort    = tf.sort(   eigenvalues, direction='DESCENDING')
    eigenvalues_argsort = tf.argsort(eigenvalues, direction='DESCENDING')
    eigenvectors_sort   = tf.gather(eigenvectors, eigenvalues_argsort, axis = 1)

    q = sum(eigenvalues_sort.numpy()<0)
    p = eigenvalues_sort.shape[0] - q

    return p, q, tf.einsum("ni,i->ni", tf.cast(eigenvectors_sort, dtype = tf.float32), tf.cast(tf.sqrt(tf.abs(eigenvalues_sort)), dtype = tf.float32))

In [7]:
d = 2
p, q, hatX = d_spectral_tensorly(A, d)

newsimulator = GRDPG(p, q, N)

hatXminus = hatX*tf.reshape(newsimulator.I, (1, newsimulator.p+newsimulator.q))

hatM = tf.einsum("ik,jk->ij", hatXminus, hatX)

M = tf.einsum("nj,kj->nk", tf.einsum("nk,kj->nj", E, B), E)
print("the max norm of the difference between M and hatM is "+str(tf.reduce_max(tf.abs(M-hatM)).numpy()))

the max norm of the difference between M and hatM is 0.44603348


In [12]:
def likelihood_evalutation(A, hatM):    
    # extract lower triangular matrix from A, M
    Lower = tf.linalg.band_part(tf.ones([N, N]), -1, 0) -  tf.linalg.band_part(tf.ones([N, N]), 0, 0)
    LowerA = tf.boolean_mask(A, Lower)
    LowerHatM = tf.boolean_mask(hatM, Lower)
    # select 0 entries of A
    maskA = (LowerA == tf.zeros_like(LowerA))
    # check if M has 0 or 1 entries
    maskM0 = (LowerHatM == tf.zeros_like(LowerHatM))
    maskM1 = (LowerHatM == tf.ones_like(LowerHatM))
    if (tf.math.reduce_any(maskM0 & ~maskA) | tf.math.reduce_any(maskM1 & maskA)):
        reject = True
        log_likelihood = 0
        break
    else:
        reject = False
        edges = tf.concat([tf.math.log(1-tf.boolean_mask(LowerHatM, maskA &(~maskM1))), 
                            tf.math.log(tf.boolean_mask(LowerHatM, ~maskA))], -1)
        log_likelihood = tf.reduce_sum(edges).numpy()

SyntaxError: 'break' outside loop (4054274593.py, line 14)

In [11]:
reject
log_likelihood

-1691.7991